# Create NA Plants Fixed Voucher

## Input:

- Path to Excel file containing claims with TPLs
- start and end voucher dates
- honda/acura hourly labor rates
- part cost factor
- handling cost factor

## Output:

- Excel voucher containing NA Plants' fixed voucher

In [1]:
from datetime import datetime
from pathlib import Path
from win10toast import ToastNotifier
import numpy as np
import pandas as pd
import os
import os.path as op
import pyodbc
import scrapbook as sb
pd.options.display.max_columns=100
pd.options.display.max_rows=1000
pd.options.display.float_format = '{:20,.2f}'.format

### Parameters needed to generate voucher:

In [2]:
start_voucher_date = '2018-09-01'
end_voucher_date = '2018-09-30'
honda_hourly_labor_rate = 120
acura_hourly_labor_rate = 141
part_cost_factor = 0.9
handling_cost_factor = 0.63

### Get current working directory:

In [3]:
cwd = os.getcwd()
cwd

'D:\\jupyter\\projects\\ZF_Automation\\papermill\\prod'

In [5]:
nb = sb.read_notebook(cwd + '\\output\\CLA_Claims_With_or_Without_TPLs.ipynb')

# Run this instead for testing purposes outside of using papermill
# nb = sb.read_notebook(cwd + '\\1_Obtain_CLA_Claims_With_or_Without_TPLs.ipynb')

In [6]:
nb.scraps['path_to_fixed_claims'].data

'\\\\mmpapp02\\mq_db\\wrp\\ZF_ATM_WRP_Project\\automated_vouchers\\outputs\\excel\\NorthAmericanPlants\\Fixed\\2019-04-23\\CLA_Claims_Fixed.xlsx'

In [7]:
claims_without_tpl = pd.read_excel(nb.scraps['path_to_fixed_claims'].data, index=False, dtype={'MODEL_YEAR': str, 'REPLACED_PART5': str},
                                  parse_dates=['SUBMITTED_DATE','REOPENED_SUBMITTED_DATE','REPAIR_ORDER_DATE',
                                               'AF_OFF_DATE','ENGINE_BUILD_DATE'])

In [8]:
claims_without_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM
0,5FRYD4H91GB001010251481035771,201705,5FRYD4H91GB001010,5FRYD4H91GB001010,2017-06-16,2017-08-07,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,676.00,062005NCA79RM,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,~,~,NaN,NaN,251481,035771,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,2017-04-25,2017-05-11,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,1.20,2017-06-14 11:04:53.493,nan,nan,18759,$,"2,872.37","1,062.20","1,152.14","5,086.71",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011360,ACURA,2016,HMA,MDX,9,200215NC,7.10,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","722,113.00",1
1,5FRYD4H97GB001027251144812311,201711,5FRYD4H97GB001027,5FRYD4H97GB001027,2017-11-22,2018-01-16,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,517.00,062005NCA79RM,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,~,~,NaN,NaN,251144,812311,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,2017-10-30,2017-11-15,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,3.10,2017-11-20 12:51:31.443,nan,nan,51221,$,"2,881.81","1,520.00","2,572.40","6,974.21",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011388,ACURA,2016,HMA,MDX,9,200215NC,9.00,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","730,732.00",1
2,5FRYD4H98GB001022251482939970,201711,5FRYD4H98GB001022,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005NCA79RM,NaN,~,~,NaN,NaN,251482,939970,"WHEN DRIVING 70 MPH WHINNING NOISE,NOISE ON AC...",2017-10-02,2017-11-22,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,2.60,NaT,nan,nan,63823,$,"3,242.38","1,213.00","1,376.18","5,831.56",N,3214,Normal,1,06200,062005NCA79RM ...,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011324,ACURA,2016,HMA,MDX,9,200215NC,8.50,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1
3,5FRYD3H47GB001009251004114162,201808,5FRYD3H47GB001009,5FRYD3H47GB001009,2018-09-10,2018-10-23,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,225.00,062005NBA58RM,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,~,~,NaN,NaN,251004,114162,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,2018-07-24,2018-08-14,2015-02-02,2015-01-31,218102,101,15A,KA,5.90,125.00,-5.90,2018-09-06 14:38:06.150,nan,nan,57214,$,"-2,854.78","-1,256.50","-1,159.82","-5,271.10",N,99900,Debit,-1,06200,062005NBA58RM ...,WARRANTY A/T KIT,1,GTZ5,AB8,,Q5NB0053388,ACURA,2016,HMA,MDX,9,200215NB,0.00,1.00,"2,969.56","2,435.04","2,672.60","4,870.08","2,672.60","743,164.00",1
4,5FRYD3H47GB001009251004114162,201808,5FRYD3H47GB001009,5FRYD3H47GB001009,2018-09-10,2018-10-23,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,225.00,062005NBA58RM,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU

In [9]:
claims_without_tpl.shape

(4399, 71)

### Obtain fixed percentages stored in an MS Access database:

In [10]:
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=\\mmpapp02\mq_db\wrp\ZF_ATM_WRP_Project\version4\databases\WRP_DAT_ZF.accdb;'
)
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

sql = """
SELECT
    MODEL_YEAR,
    SHORT_PART_NO,
    SUPP_FIXED_PERC,
    ANALYSIS_END_MTH

FROM
    tbl_Fixed_Percentages_Yr_Part 
"""

try:
    fixed_percentages = pd.read_sql(sql, cnxn)

    # Close connections
    cursor.close()
    cnxn.close()
    print(sql)
except:
    print("Error connecting to database")
    cursor.close()
    cnxn.close()


SELECT
    MODEL_YEAR,
    SHORT_PART_NO,
    SUPP_FIXED_PERC,
    ANALYSIS_END_MTH

FROM
    tbl_Fixed_Percentages_Yr_Part 



In [11]:
fixed_percentages

,MODEL_YEAR,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH
0,2015,06200,0.53,201806
1,2015,06224,0.88,201806
2,2015,06225,0.91,201806
3,2015,06231,1.00,201806
4,2015,06233,0.97,201806
5,2015,06235,0.80,201806
6,2015,06280,0.34,201806
7,2016,06200,0.93,201806
8,2016,06221,0.50,201806
9,2016,06222,1.00,201806


### Add ```SUPP_FIXED_PERC``` column by LEFT JOIN-ing to the fixed percentages data set:

In [12]:
claims_without_tpl = pd.merge(claims_without_tpl, fixed_percentages, how='left', left_on=['MODEL_YEAR', 'REPLACED_PART5'],
                          right_on=['MODEL_YEAR', 'SHORT_PART_NO'], indicator=True)

In [13]:
claims_without_tpl[['MODEL_YEAR','REPLACED_PART5','SUPP_FIXED_PERC']].head()

,MODEL_YEAR,REPLACED_PART5,SUPP_FIXED_PERC
0,2016,06200,0.93
1,2016,06200,0.93
2,2016,06200,0.93
3,2016,06200,0.93
4,2016,06200,0.93


In [14]:
claims_without_tpl['SUPP_FIXED_PERC'].value_counts()

0.93    1787
0.53    1535
0.60     352
0.34     187
0.99     100
0.97      60
0.94      50
0.88      30
1.00      25
0.92      23
0.97      22
0.80       8
0.91       5
0.50       3
Name: SUPP_FIXED_PERC, dtype: int64

In [15]:
claims_without_tpl.to_excel(r'D:\temp\fixed_claims.xlsx', index=False)

### Add WRPID column which is in the format: WRP219F + 2-digit month + 2-digit year where month/year depends on end voucher date

In [16]:
year = end_voucher_date[2:4]
year

'18'

In [17]:
month = end_voucher_date[5:7]
month

'09'

In [18]:
claims_without_tpl = claims_without_tpl.assign(WRPID='WRP219F' + month + year)

#### Let's confirm that ```WRPID``` has been created (scroll to far right):

In [19]:
claims_without_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,_merge,WRPID
0,5FRYD4H91GB001010251481035771,201705,5FRYD4H91GB001010,5FRYD4H91GB001010,2017-06-16,2017-08-07,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,676.00,062005NCA79RM,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,~,~,NaN,NaN,251481,035771,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,2017-04-25,2017-05-11,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,1.20,2017-06-14 11:04:53.493,nan,nan,18759,$,"2,872.37","1,062.20","1,152.14","5,086.71",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011360,ACURA,2016,HMA,MDX,9,200215NC,7.10,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","722,113.00",1,06200,0.93,201806,both,WRP219F0918
1,5FRYD4H97GB001027251144812311,201711,5FRYD4H97GB001027,5FRYD4H97GB001027,2017-11-22,2018-01-16,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,517.00,062005NCA79RM,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,~,~,NaN,NaN,251144,812311,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,2017-10-30,2017-11-15,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,3.10,2017-11-20 12:51:31.443,nan,nan,51221,$,"2,881.81","1,520.00","2,572.40","6,974.21",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011388,ACURA,2016,HMA,MDX,9,200215NC,9.00,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","730,732.00",1,06200,0.93,201806,both,WRP219F0918
2,5FRYD4H98GB001022251482939970,201711,5FRYD4H98GB001022,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005NCA79RM,NaN,~,~,NaN,NaN,251482,939970,"WHEN DRIVING 70 MPH WHINNING NOISE,NOISE ON AC...",2017-10-02,2017-11-22,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,2.60,NaT,nan,nan,63823,$,"3,242.38","1,213.00","1,376.18","5,831.56",N,3214,Normal,1,06200,062005NCA79RM ...,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011324,ACURA,2016,HMA,MDX,9,200215NC,8.50,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,both,WRP219F0918
3,5FRYD3H47GB001009251004114162,201808,5FRYD3H47GB001009,5FRYD3H47GB001009,2018-09-10,2018-10-23,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,225.00,062005NBA58RM,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,~,~,NaN,NaN,251004,114162,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,2018-07-24,2018-08-14,2015-02-02,2015-01-31,218102,101,15A,KA,5.90,125.00,-5.90,2018-09-06 14:38:06.150,nan,nan,57214,$,"-2,854.78","-1,256.50","-1,159.82","-5,271.10",N,99900,Debit,-1,06200,062005NBA58RM ...,WARRANTY A/T KIT,1,GTZ5,AB8,,Q5NB0053388,ACURA,2016,HMA,MDX,9,200215NB,0.00,1.00,"2,969.56","2,435.04","2,672.60","4,870.08","2,672.60","743,164.00",1,06200,0.93,201806,both,WRP219F0918
4,5FRYD3H47GB001009251004114162,201808

### Add ```HRLY_RATE_USD``` column:

In [20]:
claims_without_tpl['HRLY_RATE_USD'] = np.where(claims_without_tpl['NAMEPLATE'] == 'ACURA', int(acura_hourly_labor_rate), int(honda_hourly_labor_rate))

#### Let's confirm that ```HRLY_RATE_USD``` column has been created (scroll to far right):

In [21]:
claims_without_tpl.query("NAMEPLATE == 'ACURA'").head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,_merge,WRPID,HRLY_RATE_USD
0,5FRYD4H91GB001010251481035771,201705,5FRYD4H91GB001010,5FRYD4H91GB001010,2017-06-16,2017-08-07,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,676.00,062005NCA79RM,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,~,~,NaN,NaN,251481,035771,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,2017-04-25,2017-05-11,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,1.20,2017-06-14 11:04:53.493,nan,nan,18759,$,"2,872.37","1,062.20","1,152.14","5,086.71",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011360,ACURA,2016,HMA,MDX,9,200215NC,7.10,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","722,113.00",1,06200,0.93,201806,both,WRP219F0918,141
1,5FRYD4H97GB001027251144812311,201711,5FRYD4H97GB001027,5FRYD4H97GB001027,2017-11-22,2018-01-16,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,517.00,062005NCA79RM,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,~,~,NaN,NaN,251144,812311,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,2017-10-30,2017-11-15,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,3.10,2017-11-20 12:51:31.443,nan,nan,51221,$,"2,881.81","1,520.00","2,572.40","6,974.21",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011388,ACURA,2016,HMA,MDX,9,200215NC,9.00,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","730,732.00",1,06200,0.93,201806,both,WRP219F0918,141
2,5FRYD4H98GB001022251482939970,201711,5FRYD4H98GB001022,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005NCA79RM,NaN,~,~,NaN,NaN,251482,939970,"WHEN DRIVING 70 MPH WHINNING NOISE,NOISE ON AC...",2017-10-02,2017-11-22,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,2.60,NaT,nan,nan,63823,$,"3,242.38","1,213.00","1,376.18","5,831.56",N,3214,Normal,1,06200,062005NCA79RM ...,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011324,ACURA,2016,HMA,MDX,9,200215NC,8.50,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,both,WRP219F0918,141
3,5FRYD3H47GB001009251004114162,201808,5FRYD3H47GB001009,5FRYD3H47GB001009,2018-09-10,2018-10-23,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,225.00,062005NBA58RM,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,~,~,NaN,NaN,251004,114162,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,2018-07-24,2018-08-14,2015-02-02,2015-01-31,218102,101,15A,KA,5.90,125.00,-5.90,2018-09-06 14:38:06.150,nan,nan,57214,$,"-2,854.78","-1,256.50","-1,159.82","-5,271.10",N,99900,Debit,-1,06200,062005NBA58RM ...,WARRANTY A/T KIT,1,GTZ5,AB8,,Q5NB0053388,ACURA,2016,HMA,MDX,9,200215NB,0.00,1.00,"2,969.56","2,435.04","2,672.60","4,870.08","2,672.60","743,164.00",1,06200,0.93,201806,both,WRP219F0918,141
4,5FRYD3

In [22]:
claims_without_tpl.query("NAMEPLATE == 'HONDA'").head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,_merge,WRPID,HRLY_RATE_USD
1390,5FNYF6H97GB001097207130460882,201702,5FNYF6H97GB001097,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062805EZA08,NaN,~,~,NaN,NaN,207130,460882,"MODULE, TRANSMISSION CONTROL - REPLACE",2017-01-19,2017-01-30,2015-05-07,2015-05-06,2231A0,101,15A,KA,0.60,0.00,0.00,NaT,nan,nan,21590,$,739.28,61.80,"1,085.27","1,886.35",N,3214,Normal,1,06280,062805EZA08,"CONTROL UNIT, TRANS",1,GTG7,AK9,,Q5NC0045632,HONDA,2016,HMA,PILOT,9,200215NC,0.60,1.00,430.63,110.76,387.57,221.52,221.52,nan,1,06280,0.60,201806,both,WRP219F0918,120
1391,5FNYF6H08GB002292208470967921,201806,5FNYF6H08GB002292,5FNYF6H08GB002292,2018-06-18,2018-08-07,2019-04-03,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,309.00,062015EZA05,"TRANSMISSION SLIPS IN 1ST GEAR FRM STOP, WORSE...",~,~,NaN,NaN,208470,967921,"TRANSMISSION SLIPS IN 1ST GEAR FRM STOP, WORSE...",2018-04-12,2018-06-01,2015-05-18,2015-05-16,218102,101,15A,KA,6.80,125.00,2.30,2018-06-14 13:27:16.233,nan,nan,59214,$,"3,052.03","1,207.90","2,260.69","6,520.62",N,504,Normal,1,06200,062005EZA09RM ...,WARRANTY A/T KIT,1,GTG7,AL9,,Q5NC0048277,HONDA,2016,HMA,PILOT,9,200215NC,9.10,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","739,805.00",1,06200,0.93,201806,both,WRP219F0918,120
1392,5FNYF6H93GB002506206528142355,201809,5FNYF6H93GB002506,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062015EZA05,NaN,~,~,NaN,NaN,206528,142355,CUSTOMER STATES THAT THE CAR IS MAKING A SOUND...,2018-08-03,2018-09-11,2015-05-20,2015-05-19,218102,101,101,KA,6.80,125.00,1.70,NaT,nan,nan,43535,$,"2,927.53","1,255.50","2,585.57","6,768.60",N,504,Normal,1,06200,062005EZA09RM ...,WARRANTY A/T KIT,1,GTG7,AK9,,Q5NC0049611,HONDA,2016,HMA,PILOT,9,200215NC,8.50,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,both,WRP219F0918,120
1393,5FNYF6H93GB002506206528142355,201902,5FNYF6H93GB002506,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062015EZA05,NaN,~,~,NaN,NaN,206528,142355,CUSTOMER STATES THAT THE CAR IS MAKING A SOUND...,2018-08-03,2018-09-11,2015-05-20,2015-05-19,218102,101,101,KA,6.80,125.00,-6.80,NaT,nan,nan,43535,$,"-1,902.89","-1,255.50","-2,585.57","-5,743.96",N,99900,Debit,-1,06200,062005EZA09RM ...,WARRANTY A/T KIT,1,GTG7,AK9,,Q5NC0049611,HONDA,2016,HMA,PILOT,9,200215NC,0.00,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,both,WRP219F0918,120
1394,5FNYF6H0XGB002522208195345660,201708,5FNYF6H0XGB002522,5FNYF6H0XGB002522,2017-08-25,2017-10-10,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,235.00",KIT,nan,606.00,062355L9000,CUSTOMER STATES SOMETHING LEAKING UNDER ENGINE...,~,~,NaN,NaN,208195,345660,CUSTOMER STATES SOMETHING LEAKING UNDER ENGINE...,2017-08-09,2017

### Add calculated costs:

#### Minimum part cost (lesser of: 2 x FOB or 90% dealer net) was already defined in the main SQL query:

In [23]:
claims_without_tpl = claims_without_tpl.assign(CALCULATED_PART_COST_USD=claims_without_tpl['MIN_PART_COST_USD'])

#### Calculated flat rate labor cost (USD) is just the FLAT_RATE_HRS_QTY x nameplate hourly labor rate:

np.where() function is equivalent to Excel's =if(condition, else, etc) function: if nameplate = Acura, then use the acura labor rate x flat rate hours, etc

In [24]:
claims_without_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] = np.where(claims_without_tpl['NAMEPLATE'] == 'ACURA',
                                                        claims_without_tpl['FLAT_RATE_HRS_QTY'] * int(acura_hourly_labor_rate),
                                                        claims_without_tpl['FLAT_RATE_HRS_QTY'] * int(honda_hourly_labor_rate))

#### Calculated diagnostic labor cost (USD) = (claim labor hours x hourly labor rate) - (flat rate hours x hourly labor rate)

In [25]:
claims_without_tpl = claims_without_tpl.assign(CALCULATED_DIAGNOSTIC_LABOR_COST_USD=(claims_without_tpl['CLAIM_LABOR_HRS_QTY'] *
                               claims_without_tpl['HRLY_RATE_USD']) - (claims_without_tpl['FLAT_RATE_HRS_QTY'] *
                               claims_without_tpl['HRLY_RATE_USD']))

#### Calculated handling cost = dealer net part cost x handling cost factor:

In [26]:
claims_without_tpl = claims_without_tpl.assign(CALCULATED_HDLG_CHG_AMT_USD=claims_without_tpl['DLR_NET_PRCE_AMT_USD'] *
                               float(handling_cost_factor))

#### Calculated total charge = part cost + labor cost + handling cost + sublet/freight/tax (CLA labor - WHA labor)

In [27]:
claims_without_tpl['CALCULATED_TOTAL_COST_USD'] = np.where(claims_without_tpl['ACTUAL_LABOR_CHG_AMT_USD'] == 0,
            claims_without_tpl['CALCULATED_PART_COST_USD'] + claims_without_tpl['CALCULATED_HDLG_CHG_AMT_USD'],
            claims_without_tpl['CALCULATED_PART_COST_USD'] + claims_without_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] +
            claims_without_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'] + claims_without_tpl['CALCULATED_HDLG_CHG_AMT_USD'] +
                                                       claims_without_tpl['FRGT_SUBL_TAX_AMT_USD']
        )

#### Calculated total labor cost (USD) = calculated flat rate labor cost + calculated diagnostic labor cost:

In [28]:
claims_without_tpl = claims_without_tpl.assign(CALCULATED_TOTAL_LABOR_COST_USD=claims_without_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] +
                               claims_without_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'])

### Add ```ROW_NUM``` column to identify duplicate records due to claim adjustments:

In [29]:
claims_without_tpl['ROW_NUM'] = claims_without_tpl.sort_values(by=['RECORD_DESC'], ascending=False) \
                                               .groupby(['VIN_DLR_CLM_SK']).cumcount() + 1

In [30]:
claims_without_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,_merge,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM
0,5FRYD4H91GB001010251481035771,201705,5FRYD4H91GB001010,5FRYD4H91GB001010,2017-06-16,2017-08-07,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,676.00,062005NCA79RM,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,~,~,NaN,NaN,251481,035771,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,2017-04-25,2017-05-11,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,1.20,2017-06-14 11:04:53.493,nan,nan,18759,$,"2,872.37","1,062.20","1,152.14","5,086.71",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011360,ACURA,2016,HMA,MDX,9,200215NC,7.10,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","722,113.00",1,06200,0.93,201806,both,WRP219F0918,141,"2,249.69",831.90,169.20,"1,574.78","4,950.57","1,001.10",1
1,5FRYD4H97GB001027251144812311,201711,5FRYD4H97GB001027,5FRYD4H97GB001027,2017-11-22,2018-01-16,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,517.00,062005NCA79RM,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,~,~,NaN,NaN,251144,812311,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,2017-10-30,2017-11-15,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,3.10,2017-11-20 12:51:31.443,nan,nan,51221,$,"2,881.81","1,520.00","2,572.40","6,974.21",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011388,ACURA,2016,HMA,MDX,9,200215NC,9.00,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","730,732.00",1,06200,0.93,201806,both,WRP219F0918,141,"2,249.69",831.90,437.10,"1,574.78","5,218.47","1,269.00",1
2,5FRYD4H98GB001022251482939970,201711,5FRYD4H98GB001022,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005NCA79RM,NaN,~,~,NaN,NaN,251482,939970,"WHEN DRIVING 70 MPH WHINNING NOISE,NOISE ON AC...",2017-10-02,2017-11-22,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,2.60,NaT,nan,nan,63823,$,"3,242.38","1,213.00","1,376.18","5,831.56",N,3214,Normal,1,06200,062005NCA79RM ...,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011324,ACURA,2016,HMA,MDX,9,200215NC,8.50,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,both,WRP219F0918,141,"2,249.69",831.90,366.60,"1,574.78","5,147.97","1,198.50",1
3,5FRYD3H47GB001009251004114162,201808,5FRYD3H47GB001009,5FRYD3H47GB001009,2018-09-10,2018-10-23,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,225.00,062005NBA58RM,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,~,~,NaN,NaN,251004,114162,WHEN BRAKING FROM SPEED AND ACCELERATING TO TU...,2018-07-24,2018-08-14,2015-02-02,20

In [31]:
duplicate_rows = claims_without_tpl[claims_without_tpl.duplicated(subset=['VIN_DLR_CLM_SK'], keep=False)]
duplicate_rows.sort_values(['VIN_DLR_CLM_SK', 'ROW_NUM'])[:5]

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,_merge,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM
2932,19UUB2F30FA006782251523020241,201703,19UUB2F30FA006782,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062245J4000,NaN,~,~,NaN,NaN,251523,020241,COOLER AND PIPE KIT REPLACE AT SAME TIME TRANS...,2017-02-10,2017-02-24,2014-09-20,2014-09-19,218097,101,160,KA,0.00,0.00,0.00,NaT,nan,nan,21902,$,133.74,0.00,82.30,216.04,N,3214,Normal,1,06224,062245J4000,COOLER & PIPES KIT,1,FTZ3,AD6,,Q5J40028624,ACURA,2015,MAP,TLX,9,200215J4,0.00,1.00,294.87,147.42,265.38,294.84,265.38,nan,1,06224,0.88,201806,both,WRP219F0918,141,265.38,0.00,0.00,185.77,451.15,0.00,1
2933,19UUB2F30FA006782251523020241,201703,19UUB2F30FA006782,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062245J4000,NaN,~,~,NaN,NaN,251523,020241,COOLER AND PIPE KIT REPLACE AT SAME TIME TRANS...,2017-02-10,2017-02-24,2014-09-20,2014-09-19,218097,101,160,KA,0.00,0.00,0.00,NaT,nan,nan,21902,$,-133.74,0.00,-82.30,-216.04,N,99900,Debit,-1,06224,062245J4000,COOLER & PIPES KIT,1,FTZ3,AD6,,Q5J40028624,ACURA,2015,MAP,TLX,9,200215J4,0.00,1.00,294.87,147.42,265.38,294.84,265.38,nan,1,06224,0.88,201806,both,WRP219F0918,141,265.38,0.00,0.00,185.77,451.15,0.00,2
2931,19UUB2F30FA006782251523020241,201705,19UUB2F30FA006782,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062245J4000,NaN,~,~,NaN,NaN,251523,020241,COOLER AND PIPE KIT REPLACE AT SAME TIME TRANS...,2017-02-10,2017-02-24,2014-09-20,2014-09-19,218097,101,160,KA,0.00,0.00,0.00,NaT,nan,nan,21902,$,185.18,0.00,82.30,267.48,N,55200,Adjustment,0,06224,062245J4000,COOLER & PIPES KIT,1,FTZ3,AD6,,Q5J40028624,ACURA,2015,MAP,TLX,9,200215J4,0.00,1.00,294.87,147.42,265.38,294.84,265.38,nan,1,06224,0.88,201806,both,WRP219F0918,141,265.38,0.00,0.00,185.77,451.15,0.00,3
3995,19UUB2F30FA020889251424895811,201807,19UUB2F30FA020889,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005J4A71RM,NaN,~,~,NaN,NaN,251424,895811,CUSTOMER STATES A WEEK AGO WHILE DRIVING AT AB...,2018-07-09,2018-07-23,2015-03-18,2015-03-16,218102,101,101,KA,5.80,0.00,1.00,NaT,nan,nan,21119,$,"2,769.59",884.00,"2,109.87","5,763.46",N,3217,Normal,1,06200,062005J4A71RM ...,WARRANTY A/T KIT,1,FTZ3,AD6,,Q5J40057270,ACURA,2015,MAP,TLX,9,200215J4,6.80,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",nan,1,06200,0.53,201806,both,WRP219F0918,141,"2,670.41",817.80,141.00,"1,869.29","5,498.50",958.80,1
3999,19UUB2F30FA020889251424895811,201902,19UUB2F30FA020889,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005J4A71RM,NaN,~,~,NaN,NaN,251424,895811,CUSTOMER STATES A WEEK AGO WHILE DRIVING AT AB...,2018-07-09,201

### Add actual claim level total cost:

In [32]:
pivot = pd.pivot_table(claims_without_tpl, index='VIN_DLR_CLM_SK', values='ACTUAL_TOTAL_CHG_AMT_USD', aggfunc='sum')

In [33]:
pivot.head()

,ACTUAL_TOTAL_CHG_AMT_USD
VIN_DLR_CLM_SK,
19UUB2F30FA000593251022430801,"6,591.38"
19UUB2F30FA001193251507945736,"6,650.06"
19UUB2F30FA001579251047572293,"5,772.37"
19UUB2F30FA001629251004877251,"5,132.61"
19UUB2F30FA002795251480910812,"1,269.66"


In [34]:
pivot.columns = ['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD']
pivot.reset_index(level=0, inplace=True)
pivot.head()

,VIN_DLR_CLM_SK,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD
0,19UUB2F30FA000593251022430801,"6,591.38"
1,19UUB2F30FA001193251507945736,"6,650.06"
2,19UUB2F30FA001579251047572293,"5,772.37"
3,19UUB2F30FA001629251004877251,"5,132.61"
4,19UUB2F30FA002795251480910812,"1,269.66"


In [35]:
claims_without_tpl = pd.merge(claims_without_tpl, pivot, how='left', left_on='VIN_DLR_CLM_SK', right_on='VIN_DLR_CLM_SK')

In [36]:
claims_without_tpl.shape

(4399, 85)

In [37]:
duplicate_rows = claims_without_tpl[claims_without_tpl.duplicated(subset=['VIN_DLR_CLM_SK'], keep=False)]
duplicate_rows.sort_values(['VIN_DLR_CLM_SK', 'ROW_NUM'])[['VIN_DLR_CLM_SK','ROW_NUM','ACTUAL_TOTAL_CHG_AMT_USD',
                                                           'ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD']][:10]

,VIN_DLR_CLM_SK,ROW_NUM,ACTUAL_TOTAL_CHG_AMT_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD
2932,19UUB2F30FA006782251523020241,1,216.04,267.48
2933,19UUB2F30FA006782251523020241,2,-216.04,267.48
2931,19UUB2F30FA006782251523020241,3,267.48,267.48
3995,19UUB2F30FA020889251424895811,1,"5,763.46",969.36
3999,19UUB2F30FA020889251424895811,2,"-4,794.10",969.36
3595,19UUB2F32FA015354251543080773,1,"5,393.73",969.34
3597,19UUB2F32FA015354251543080773,2,"-4,424.39",969.34
3763,19UUB2F32FA017802251424909971,1,"5,865.46",969.36
3764,19UUB2F32FA017802251424909971,2,"-4,896.10",969.36
4017,19UUB2F32FA021400251049128045,1,"7,042.95",948.81


### Obtain ASN receive dates

In [38]:
username_m = os.environ['windowsuser']
password_m = os.environ['mainframepwd']

In [39]:
%%time
cnxn_string = 'DSN=DSSOGW01;UID=' + username_m + ';PWD=' + password_m

cnxn = pyodbc.connect(cnxn_string)
cursor = cnxn.cursor()

sql = """
SELECT
    RTRIM(TRANS_SERIAL_NO) AS TRANS_SERIAL_NO,
    ASN_RECEIVE_DATE

FROM
    WAR.WRTRN1
    
WHERE
    TRANS_SERIAL_NO LIKE 'Q%'
"""

try:
    print("Obtaining ASN records 20,000 rows at a time...")
    asn = pd.DataFrame()
    for chunk in pd.read_sql(sql, cnxn, index_col=None, parse_dates=['ASN_RECEIVE_DATE'], chunksize=20000):
        asn = pd.concat([asn, chunk])

    # Close connections
    cursor.close()
    cnxn.close()
except:
    print("Error connecting to ASN database server")
    cursor.close()
    cnxn.close()
print("Finished obtainig ASN records")

Obtaining ASN records 20,000 rows at a time...
Finished obtainig ASN records
Wall time: 48.1 s


In [40]:
asn.shape

(998485, 2)

#### LEFT JOIN the asn data to the claims without TPL data:

In [41]:
claims_without_tpl = pd.merge(claims_without_tpl, asn, how='left', left_on='TRANS_SERIAL_NO', right_on='TRANS_SERIAL_NO')

In [42]:
claims_without_tpl.shape

(4399, 86)

In [43]:
claims_without_tpl.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,_merge,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE
0,5FRYD4H91GB001010251481035771,201705,5FRYD4H91GB001010,5FRYD4H91GB001010,2017-06-16,2017-08-07,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,676.00,062005NCA79RM,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,~,~,NaN,NaN,251481,035771,SQUEAK UPON ACCELERATE; 30MPH TO A SLOW DOWN T...,2017-04-25,2017-05-11,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,1.20,2017-06-14 11:04:53.493,nan,nan,18759,$,"2,872.37","1,062.20","1,152.14","5,086.71",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011360,ACURA,2016,HMA,MDX,9,200215NC,7.10,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","722,113.00",1,06200,0.93,201806,both,WRP219F0918,141,"2,249.69",831.90,169.20,"1,574.78","4,950.57","1,001.10",1,"5,086.71",2015-01-22
1,5FRYD4H97GB001027251144812311,201711,5FRYD4H97GB001027,5FRYD4H97GB001027,2017-11-22,2018-01-16,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,517.00,062005NCA79RM,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,~,~,NaN,NaN,251144,812311,AFTER PERFORMING 17-026 FOUND NO CHANGE IN NOI...,2017-10-30,2017-11-15,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,3.10,2017-11-20 12:51:31.443,nan,nan,51221,$,"2,881.81","1,520.00","2,572.40","6,974.21",N,3214,Normal,1,06200,062005NCA79RM,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011388,ACURA,2016,HMA,MDX,9,200215NC,9.00,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","730,732.00",1,06200,0.93,201806,both,WRP219F0918,141,"2,249.69",831.90,437.10,"1,574.78","5,218.47","1,269.00",1,"6,974.21",2015-01-22
2,5FRYD4H98GB001022251482939970,201711,5FRYD4H98GB001022,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005NCA79RM,NaN,~,~,NaN,NaN,251482,939970,"WHEN DRIVING 70 MPH WHINNING NOISE,NOISE ON AC...",2017-10-02,2017-11-22,2015-01-21,2015-01-20,218102,101,15A,KA,5.90,125.00,2.60,NaT,nan,nan,63823,$,"3,242.38","1,213.00","1,376.18","5,831.56",N,3214,Normal,1,06200,062005NCA79RM ...,WARRANTY A/T KIT,1,GTZ6,AE5,,Q5NC0011324,ACURA,2016,HMA,MDX,9,200215NC,8.50,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,both,WRP219F0918,141,"2,249.69",831.90,366.60,"1,574.78","5,147.97","1,198.50",1,"5,831.56",2015-01-22
3,5FRYD3H47GB001009251004114162,201808,5FRYD3H47GB001009,5FRYD3H47GB001009,2018-09-10,2018-10-23,2019-03-25,NaT,nan,REOPENED_FOR_ANALYSIS,OPEN,"6,200.00",TRANSMISSION,nan,225.00,062005NBA58RM,WHEN BRAKING FROM SPEED AND ACCELERATING 

### Number of rows with missing ASN_RECEIVE_DATE

In [44]:
# Number of claims with missing ASN_RECEIVE_DATE
claims_without_tpl[claims_without_tpl['ASN_RECEIVE_DATE'].isna()].shape[0]

95

### Fill missing ASN_RECEIVE_DATE with engine build date minus 4 days:

In [45]:
claims_without_tpl['ASN_RECEIVE_DATE'].fillna(value=claims_without_tpl['ENGINE_BUILD_DATE'] - pd.Timedelta('4 days'), inplace=True)

In [46]:
# Number of claims with missing ASN_RECEIVE_DATE
claims_without_tpl[claims_without_tpl['ASN_RECEIVE_DATE'].isna()].shape[0]

0

In [47]:
assert claims_without_tpl[claims_without_tpl['ASN_RECEIVE_DATE'].isna()].shape[0] == 0

### With ```ASN_RECEIVE_DATE``` fully populated, we can now create ```TransDTF``` column = RO date minus ASN receive date:

In [48]:
claims_without_tpl = claims_without_tpl.assign(TransDTF=(claims_without_tpl['REPAIR_ORDER_DATE']
                                         - claims_without_tpl['ASN_RECEIVE_DATE']).apply(lambda x: x.days))

In [49]:
claims_without_tpl[['REPAIR_ORDER_DATE','ASN_RECEIVE_DATE','TransDTF']].head()

,REPAIR_ORDER_DATE,ASN_RECEIVE_DATE,TransDTF
0,2017-04-25,2015-01-22,824
1,2017-10-30,2015-01-22,1012
2,2017-10-02,2015-01-22,984
3,2018-07-24,2015-02-06,1264
4,2018-07-24,2015-02-06,1264


### Obtain Mass Pro Cost from E-Quote tables:

In [50]:
print("Connecting to DSNOGW01 server")
cnxn_string = 'DSN=DSNOGW01;UID=' + username_m + ';PWD=' + password_m

cnxn = pyodbc.connect(cnxn_string)
cursor = cnxn.cursor()

sql = """
SELECT
    *

FROM (

SELECT
    EXREP1.NEW_STATUS,
    EXCOD1.COD_NAME,
    EXWRK1.L1_DSG_CHG_PRT_NO,
    LEFT(EXWRK1.L1_DSG_CHG_PRT_NO,8) AS PART8,
    EXPLT1.PLNT_ACRNYM,
    EXPLT1.PLNT_DESC,
    /** Identify duplicate values using row_number() function, partitioned by part8 and modified date **/
    row_number() OVER(PARTITION BY LEFT(EXWRK1.L1_DSG_CHG_PRT_NO,8), DATE(EXREP1.DATE_MODIFIED) ORDER BY EXWRK1.Q_CAL_FINAL_TOT_CS) AS ROW_NUM,
    EXWRK1.Q_CAL_FINAL_TOT_CS,
    EXQOT1.Q_EFF_START_DT,
    EXQOT1.Q_EFF_END_DT,
    DATE(EXREP1.DATE_MODIFIED) AS APPROVED_DATE

FROM
    EQX.EXWRK1 EXWRK1

    INNER JOIN EQX.EXQPL1 EXQPL1 ON
        EXWRK1.SUPLR_STG_ID = EXQPL1.SUPLR_STG_ID

    INNER JOIN EQX.EXPLT1 EXPLT1 ON
        EXPLT1.PLNT_ID = EXQPL1.PLNT_ID

    INNER JOIN EQX.EXREP1 EXREP1 ON
        EXWRK1.WRK_ID = EXREP1.WRK_ID

    INNER JOIN EQX.EXQOT1 EXQOT1 ON
        EXWRK1.SUPLR_STG_ID = EXQOT1.SUPLR_STG_ID

    LEFT JOIN EQX.EXSPP1 EXSPP1 ON
        EXWRK1.SUPLR_NO = EXSPP1.SUPLR_NO

    /** Code value can be used more than once based on COD_DOMAIN.  Only want the ones related to 'WORK_LIST_STATUS' **/
    LEFT JOIN (
        SELECT *

        FROM
            EQX.EXCOD1

        WHERE
            COD_DOMAIN = 'WORK_LIST_STATUS'
    ) EXCOD1 ON
        EXREP1.NEW_STATUS = EXCOD1.COD_VALUE

WHERE
    EXWRK1.SUPLR_NO = '518227'
    AND EXWRK1.L1_DSG_CHG_PRT_NO LIKE '200215%'
    AND EXCOD1.COD_NAME = 'Quote Approved MP'

ORDER BY
    PART8,
    DATE(EXREP1.DATE_MODIFIED),
    row_number() OVER(PARTITION BY LEFT(EXWRK1.L1_DSG_CHG_PRT_NO,8), DATE(EXREP1.DATE_MODIFIED) ORDER BY EXWRK1.Q_CAL_FINAL_TOT_CS)

) AS TEMP

WHERE
    ROW_NUM = 1
"""
try:
    print("Obtaining E-Quote records...")
    equote = pd.read_sql(sql, cnxn, index_col=None, parse_dates=['APPROVED_DATE'])
    cursor.close()
    cnxn.close()

except:
    # Close connections
    cursor.close()
    cnxn.close()
    print("Error occurred with connecting to the database")
print("Finished obtaining E-Quote mass production part costs")

Connecting to DSNOGW01 server
Obtaining E-Quote records...
Finished obtaining E-Quote mass production part costs


#### Define function to calculate mass pro cost at time of failure (RO date):

In [51]:
def getMassProCost(row):
    return equote[(equote['PART8'] == row['REPLACED_PART_EQUOTE_FMT']) & (equote['APPROVED_DATE'] ==
           equote[(equote['PART8'] == row['REPLACED_PART_EQUOTE_FMT']) & (equote['APPROVED_DATE'] <= row['REPAIR_ORDER_DATE'])]['APPROVED_DATE']
           .max())].sort_values(by=['Q_CAL_FINAL_TOT_CS'])['Q_CAL_FINAL_TOT_CS'].values[0]

#### Now, apply the function we defined above to create a new column containing the mass pro part cost at time of failure (RO Date):

In [52]:
print("Applying mass pro part cost x 205% calculations.  Please wait...")
claims_without_tpl['MassProCost_USD'] = claims_without_tpl.apply(getMassProCost, axis='columns')
claims_without_tpl = claims_without_tpl.assign(MassProCostX205_USD = claims_without_tpl['MassProCost_USD'] * 2.05)
print("Finished applying mass pro part cost x 205% calculations.")

Applying mass pro part cost x 205% calculations.  Please wait...
Finished applying mass pro part cost x 205% calculations.


In [53]:
claims_without_tpl[['REPLACED_PART_EQUOTE_FMT','REPAIR_ORDER_DATE','MassProCost_USD','MassProCostX205_USD']].head()

,REPLACED_PART_EQUOTE_FMT,REPAIR_ORDER_DATE,MassProCost_USD,MassProCostX205_USD
0,200215NC,2017-04-25,"2,173.67","4,456.02"
1,200215NC,2017-10-30,"2,088.52","4,281.47"
2,200215NC,2017-10-02,"2,088.52","4,281.47"
3,200215NB,2018-07-24,"1,962.59","4,023.31"
4,200215NB,2018-07-24,"1,962.59","4,023.31"


### 'zero out' cost amounts for duplicate rows to prevent double-charging of supplier

In [54]:
claims_without_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                                  claims_without_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'], 0)
claims_without_tpl['FLAT_RATE_HRS_QTY'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                               claims_without_tpl['FLAT_RATE_HRS_QTY'], 0)
claims_without_tpl['CALCULATED_PART_COST_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                      claims_without_tpl['CALCULATED_PART_COST_USD'], 0)
claims_without_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                                 claims_without_tpl['CALCULATED_FLAT_RATE_LABOR_COST_USD'], 0)
claims_without_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                                  claims_without_tpl['CALCULATED_DIAGNOSTIC_LABOR_COST_USD'], 0)
claims_without_tpl['CALCULATED_HDLG_CHG_AMT_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                         claims_without_tpl['CALCULATED_HDLG_CHG_AMT_USD'], 0)
claims_without_tpl['CALCULATED_TOTAL_LABOR_COST_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                             claims_without_tpl['CALCULATED_TOTAL_LABOR_COST_USD'], 0)
claims_without_tpl['CALCULATED_TOTAL_COST_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                       claims_without_tpl['CALCULATED_TOTAL_COST_USD'], 0)
claims_without_tpl['DIAGNOSTIC_LABOR_HRS_QTY'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                      claims_without_tpl['DIAGNOSTIC_LABOR_HRS_QTY'], 0)
claims_without_tpl['FRGT_SUBL_TAX_AMT_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                   claims_without_tpl['FRGT_SUBL_TAX_AMT_USD'], 0)
claims_without_tpl['MassProCost_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                             claims_without_tpl['MassProCost_USD'], 0)
claims_without_tpl['MassProCostX205_USD'] = np.where(claims_without_tpl['ROW_NUM'] == 1,
                                                 claims_without_tpl['MassProCostX205_USD'], 0)

In [55]:
duplicate_rows = claims_without_tpl[claims_without_tpl.duplicated(subset=['VIN_DLR_CLM_SK'], keep=False)]
duplicate_rows.sort_values(['VIN_DLR_CLM_SK', 'ROW_NUM'])[:5]

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,_merge,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE,TransDTF,MassProCost_USD,MassProCostX205_USD
2932,19UUB2F30FA006782251523020241,201703,19UUB2F30FA006782,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062245J4000,NaN,~,~,NaN,NaN,251523,020241,COOLER AND PIPE KIT REPLACE AT SAME TIME TRANS...,2017-02-10,2017-02-24,2014-09-20,2014-09-19,218097,101,160,KA,0.00,0.00,0.00,NaT,nan,nan,21902,$,133.74,0.00,82.30,216.04,N,3214,Normal,1,06224,062245J4000,COOLER & PIPES KIT,1,FTZ3,AD6,,Q5J40028624,ACURA,2015,MAP,TLX,9,200215J4,0.00,1.00,294.87,147.42,265.38,294.84,265.38,nan,1,06224,0.88,201806,both,WRP219F0918,141,265.38,0.00,0.00,185.77,451.15,0.00,1,267.48,2014-09-18,876,"2,047.00","4,196.35"
2933,19UUB2F30FA006782251523020241,201703,19UUB2F30FA006782,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062245J4000,NaN,~,~,NaN,NaN,251523,020241,COOLER AND PIPE KIT REPLACE AT SAME TIME TRANS...,2017-02-10,2017-02-24,2014-09-20,2014-09-19,218097,101,160,KA,0.00,0.00,0.00,NaT,nan,nan,21902,$,-133.74,0.00,-82.30,-216.04,N,99900,Debit,-1,06224,062245J4000,COOLER & PIPES KIT,1,FTZ3,AD6,,Q5J40028624,ACURA,2015,MAP,TLX,9,200215J4,0.00,1.00,294.87,147.42,265.38,294.84,265.38,nan,1,06224,0.88,201806,both,WRP219F0918,141,0.00,0.00,0.00,0.00,0.00,0.00,2,0.00,2014-09-18,876,0.00,0.00
2931,19UUB2F30FA006782251523020241,201705,19UUB2F30FA006782,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062245J4000,NaN,~,~,NaN,NaN,251523,020241,COOLER AND PIPE KIT REPLACE AT SAME TIME TRANS...,2017-02-10,2017-02-24,2014-09-20,2014-09-19,218097,101,160,KA,0.00,0.00,0.00,NaT,nan,nan,21902,$,185.18,0.00,82.30,267.48,N,55200,Adjustment,0,06224,062245J4000,COOLER & PIPES KIT,1,FTZ3,AD6,,Q5J40028624,ACURA,2015,MAP,TLX,9,200215J4,0.00,1.00,294.87,147.42,265.38,294.84,265.38,nan,1,06224,0.88,201806,both,WRP219F0918,141,0.00,0.00,0.00,0.00,0.00,0.00,3,0.00,2014-09-18,876,0.00,0.00
3995,19UUB2F30FA020889251424895811,201807,19UUB2F30FA020889,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005J4A71RM,NaN,~,~,NaN,NaN,251424,895811,CUSTOMER STATES A WEEK AGO WHILE DRIVING AT AB...,2018-07-09,2018-07-23,2015-03-18,2015-03-16,218102,101,101,KA,5.80,0.00,1.00,NaT,nan,nan,21119,$,"2,769.59",884.00,"2,109.87","5,763.46",N,3217,Normal,1,06200,062005J4A71RM ...,WARRANTY A/T KIT,1,FTZ3,AD6,,Q5J40057270,ACURA,2015,MAP,TLX,9,200215J4,6.80,1.00,"2,967.12","2,433.04","2,670.41","4,866.08","2,670.41",nan,1,06200,0.53,201806,both,WRP219F0918,141,"2,670.41",817.80,141.00,"1,869.29","5,498.50",958.80,1,969.36,2015-03-04,1223,"1

### Add total and calculated total cost amounts times FIXED percentage:

In [56]:
claims_without_tpl = claims_without_tpl.assign(ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC =
                               claims_without_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'] * claims_without_tpl['SUPP_FIXED_PERC'])

In [57]:
claims_without_tpl = claims_without_tpl.assign(CALCULATED_TOTAL_COST_USD_AFTER_PERC =
                               claims_without_tpl['CALCULATED_TOTAL_COST_USD'] * claims_without_tpl['SUPP_FIXED_PERC'])

In [58]:
claims_without_tpl[['SUPP_FIXED_PERC','ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD',
                 'ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC','CALCULATED_TOTAL_COST_USD','CALCULATED_TOTAL_COST_USD_AFTER_PERC']].head()

,SUPP_FIXED_PERC,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_COST_USD_AFTER_PERC
0,0.93,"5,086.71","4,742.07","4,950.57","4,615.16"
1,0.93,"6,974.21","6,501.69","5,218.47","4,864.91"
2,0.93,"5,831.56","5,436.46","5,147.97","4,799.18"
3,0.93,0.00,0.00,0.00,0.00
4,0.93,0.00,0.00,"5,697.72","5,311.69"


### Add initial and final WRP amount columns in USD: min(actual total cost, calculated cost, MassPro205 cost)

In [59]:
claims_without_tpl = claims_without_tpl.assign(WRP_VOUCHER_AMT_USD_INITIAL = claims_without_tpl[['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC',
                                                                       'CALCULATED_TOTAL_COST_USD_AFTER_PERC',
                                                                       'MassProCostX205_USD']].min(axis='columns'))

In [60]:
claims_without_tpl[['MODEL_YEAR','REPLACED_PART5','ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD','ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC',
                    'CALCULATED_TOTAL_COST_USD_AFTER_PERC','MassProCostX205_USD','WRP_VOUCHER_AMT_USD_INITIAL']].head()

,MODEL_YEAR,REPLACED_PART5,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC,CALCULATED_TOTAL_COST_USD_AFTER_PERC,MassProCostX205_USD,WRP_VOUCHER_AMT_USD_INITIAL
0,2016,06200,"5,086.71","4,742.07","4,615.16","4,456.02","4,456.02"
1,2016,06200,"6,974.21","6,501.69","4,864.91","4,281.47","4,281.47"
2,2016,06200,"5,831.56","5,436.46","4,799.18","4,281.47","4,281.47"
3,2016,06200,0.00,0.00,0.00,0.00,0.00
4,2016,06200,0.00,0.00,"5,311.69","4,023.31",0.00


In [61]:
claims_without_tpl = claims_without_tpl.assign(WRP_VOUCHER_AMT_USD_FINAL=claims_without_tpl['WRP_VOUCHER_AMT_USD_INITIAL'])

In [62]:
claims_without_tpl.to_excel(r'D:\temp\fixed_claims_before_filtering.xlsx', index=False)

### Create initial voucher based on the following criteria:

```
Must be normal warranty claims (campaign code=N)
and must have fixed percentage
and actual claim level total cost > 0.1
and vehicle miles <= 70000
and TransDTF <= 1460 (4 years)
and WAR_CLAIM_DATE >= voucher start date
and WAR_CLAIM_DATE <= voucher end date```

In [63]:
# 'Normal' warranty claims_without_tpl ONLY
criteria1 = claims_without_tpl['CAMPAIGN_CODE'] == 'N'

# claims with applicable fixed percentages
criteria2 = claims_without_tpl['_merge'] == 'both'

# claims with actual charge amounts
criteria3 = claims_without_tpl['ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD'] > 0.1

# claims meeting warranty limits
criteria4 = claims_without_tpl['VEH_MILEAGE'] <= 70000
criteria5 = claims_without_tpl['TransDTF'] <= 1460

# Limit to voucher period
criteria6 = claims_without_tpl['WAR_CLAIM_DATE'] >= int(start_voucher_date[:4] + start_voucher_date[5:7])
criteria7 = claims_without_tpl['WAR_CLAIM_DATE'] <= int(end_voucher_date[:4] + end_voucher_date[5:7])

In [64]:
claims_without_tpl.shape

(4399, 93)

In [65]:
voucher_fixed = claims_without_tpl[criteria1  # Normal warranty claims
                        & criteria2  # Have fixed percentages
                        & criteria3  # Have actual claim level total charge amout > 0.10
                        & criteria4  # Vehicle miles <= 70000
                        & criteria5  # TransDTF <= 1460
                        & criteria6  # WAR_CLAIM_DATE >= voucher start date
                        & criteria7]  # WAR_CLAIM_DATE <= voucher end date

In [66]:
claims_without_tpl.shape

(4399, 93)

In [67]:
voucher_fixed.shape

(184, 93)

#### Create FIXED_YES_NO

In [68]:
voucher_fixed = voucher_fixed.assign(FIXED_YES_NO = 'YES')

### Obtain already paid claims to prevent double-charging of supplier:

In [69]:
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=\\mmpapp02\mq_db\wrp\ZF_ATM_WRP_Project\version4\databases\WRP_DAT_ZF.accdb;'
)
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()

sql = """
SELECT
    *

FROM
    tbl_Already_Paid_Claims
"""

try:
    already_paid_claims = pd.read_sql(sql, cnxn)

    # Close connections
    cursor.close()
    cnxn.close()
    print(sql)
except:
    print("Error connecting to database")
    cursor.close()
    cnxn.close()


SELECT
    *

FROM
    tbl_Already_Paid_Claims



In [70]:
already_paid_claims.shape

(4459, 3)

In [71]:
del voucher_fixed['_merge']

In [72]:
voucher_fixed_final = pd.merge(voucher_fixed, already_paid_claims, how='left', left_on='VIN_DLR_CLM_SK',
                                    right_on='VIN_DLR_CLM_SK', indicator=True)

In [73]:
voucher_fixed_final.shape

(184, 96)

In [74]:
voucher_fixed_final.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE,TransDTF,MassProCost_USD,MassProCostX205_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC,CALCULATED_TOTAL_COST_USD_AFTER_PERC,WRP_VOUCHER_AMT_USD_INITIAL,WRP_VOUCHER_AMT_USD_FINAL,FIXED_YES_NO,WRP_ID,WRP_RECOVERY_COST_AFTER_PERC_USD_FINAL,_merge
0,5FRYD4H68GB001169251057775001,201809,5FRYD4H68GB001169,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062025J7A00,NaN,~,~,NaN,NaN,251057,775001,TECHLINE REF# 4149364 CUST STATES ENGINE SQUEE...,2018-08-30,2018-09-20,2015-02-02,2015-01-31,218102,101,15A,KA,5.90,125.00,4.40,NaT,nan,nan,44162,$,"2,904.63","1,587.60","2,563.92","7,056.15",N,3214,Normal,1,06200,062005NCA59RM ...,WARRANTY A/T KIT,1,GTZ6,AC7,,Q5NC0011979,ACURA,2016,HMA,MDX,9,200215NC,10.30,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,WRP219F0918,141,"2,249.69",831.90,620.40,"1,574.78","5,401.77","1,452.30",1,"7,056.15",2015-01-28,1310,"2,078.91","4,261.76","6,578.08","5,035.79","4,261.76","4,261.76",YES,NaN,nan,left_only
1,5FRYD4H45GB002049251123228858,201809,5FRYD4H45GB002049,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062355L9000,NaN,~,~,NaN,NaN,251123,228858,CUSTOMER STATES THERE IS AN LEAK FROM BOTTOM O...,2018-05-30,2018-09-20,2015-02-05,2015-02-05,2191W5,101,101,KA,1.80,0.00,0.00,NaT,nan,nan,59696,$,16.08,267.30,18.80,302.18,N,3214,Normal,1,06235,062355L9000 ...,"RING SEAL KIT, SHAFT",1,GTZ6,AB9,,Q5NC0013453,ACURA,2016,HMA,MDX,9,200215NC,1.80,1.00,24.74,8.66,22.27,17.32,17.32,nan,1,06235,0.94,201806,WRP219F0918,141,17.32,253.80,0.00,15.59,286.71,253.80,1,302.18,2015-02-04,1211,"2,070.46","4,244.45",284.40,269.84,269.84,269.84,YES,NaN,nan,left_only
2,5FRYD4H83GB501080002174280552,201809,5FRYD4H83GB501080,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005NCC79RM,NaN,~,~,NaN,NaN,2174,280552,REFERENCE LINE#1 428055 - CLICKING OR HISSING ...,2018-06-28,2018-08-28,2015-02-09,2015-02-06,218102,130,15A,KC,5.90,0.00,3.70,NaT,nan,nan,42002,C,"3,860.96",735.30,398.80,"4,995.06",N,3214,Normal,1,06200,062005NCC79RM ...,WARRANTY A/T KIT,1,GTZ6,CC9,,Q5NC0013778,ACURA,2016,HMA,MDX,9,200215NC,9.60,0.77,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,WRP219F0918,141,"2,249.69",831.90,521.70,"1,574.78","5,178.07","1,353.60",1,"4,995.06",2015-02-06,1238,"2,070.46","4,244.45","4,656.63","4,827.24","4,244.45","4,244.45",YES,NaN,nan,left_only
3,5FRYD4H4XGB006999251027265521,201809,5FRYD4H4XGB006999,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,200215NB

In [75]:
voucher_fixed_final['_merge'].value_counts()

left_only     184
both            0
right_only      0
Name: _merge, dtype: int64

### Exclude already paid claims to prevent double-charging of supplier:

In [76]:
criteria = voucher_fixed_final['_merge'] == 'left_only'
voucher_fixed_final = voucher_fixed_final[criteria]

In [77]:
voucher_fixed_final.shape

(184, 96)

### Delete redundant or unnecessary columns that came from merging with the MS Access already paid claims table

In [78]:
del voucher_fixed_final['_merge']
del voucher_fixed_final['WRP_ID']
del voucher_fixed_final['WRP_RECOVERY_COST_AFTER_PERC_USD_FINAL']

In [79]:
voucher_fixed_final.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM,SHORT_PART_NO,SUPP_FIXED_PERC,ANALYSIS_END_MTH,WRPID,HRLY_RATE_USD,CALCULATED_PART_COST_USD,CALCULATED_FLAT_RATE_LABOR_COST_USD,CALCULATED_DIAGNOSTIC_LABOR_COST_USD,CALCULATED_HDLG_CHG_AMT_USD,CALCULATED_TOTAL_COST_USD,CALCULATED_TOTAL_LABOR_COST_USD,ROW_NUM,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD,ASN_RECEIVE_DATE,TransDTF,MassProCost_USD,MassProCostX205_USD,ACTUAL_CLAIM_LEVEL_TOTAL_CHG_AMT_USD_AFTER_PERC,CALCULATED_TOTAL_COST_USD_AFTER_PERC,WRP_VOUCHER_AMT_USD_INITIAL,WRP_VOUCHER_AMT_USD_FINAL,FIXED_YES_NO
0,5FRYD4H68GB001169251057775001,201809,5FRYD4H68GB001169,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062025J7A00,NaN,~,~,NaN,NaN,251057,775001,TECHLINE REF# 4149364 CUST STATES ENGINE SQUEE...,2018-08-30,2018-09-20,2015-02-02,2015-01-31,218102,101,15A,KA,5.90,125.00,4.40,NaT,nan,nan,44162,$,"2,904.63","1,587.60","2,563.92","7,056.15",N,3214,Normal,1,06200,062005NCA59RM ...,WARRANTY A/T KIT,1,GTZ6,AC7,,Q5NC0011979,ACURA,2016,HMA,MDX,9,200215NC,10.30,1.00,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,WRP219F0918,141,"2,249.69",831.90,620.40,"1,574.78","5,401.77","1,452.30",1,"7,056.15",2015-01-28,1310,"2,078.91","4,261.76","6,578.08","5,035.79","4,261.76","4,261.76",YES
1,5FRYD4H45GB002049251123228858,201809,5FRYD4H45GB002049,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062355L9000,NaN,~,~,NaN,NaN,251123,228858,CUSTOMER STATES THERE IS AN LEAK FROM BOTTOM O...,2018-05-30,2018-09-20,2015-02-05,2015-02-05,2191W5,101,101,KA,1.80,0.00,0.00,NaT,nan,nan,59696,$,16.08,267.30,18.80,302.18,N,3214,Normal,1,06235,062355L9000 ...,"RING SEAL KIT, SHAFT",1,GTZ6,AB9,,Q5NC0013453,ACURA,2016,HMA,MDX,9,200215NC,1.80,1.00,24.74,8.66,22.27,17.32,17.32,nan,1,06235,0.94,201806,WRP219F0918,141,17.32,253.80,0.00,15.59,286.71,253.80,1,302.18,2015-02-04,1211,"2,070.46","4,244.45",284.40,269.84,269.84,269.84,YES
2,5FRYD4H83GB501080002174280552,201809,5FRYD4H83GB501080,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,062005NCC79RM,NaN,~,~,NaN,NaN,2174,280552,REFERENCE LINE#1 428055 - CLICKING OR HISSING ...,2018-06-28,2018-08-28,2015-02-09,2015-02-06,218102,130,15A,KC,5.90,0.00,3.70,NaT,nan,nan,42002,C,"3,860.96",735.30,398.80,"4,995.06",N,3214,Normal,1,06200,062005NCC79RM ...,WARRANTY A/T KIT,1,GTZ6,CC9,,Q5NC0013778,ACURA,2016,HMA,MDX,9,200215NC,9.60,0.77,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69",nan,1,06200,0.93,201806,WRP219F0918,141,"2,249.69",831.90,521.70,"1,574.78","5,178.07","1,353.60",1,"4,995.06",2015-02-06,1238,"2,070.46","4,244.45","4,656.63","4,827.24","4,244.45","4,244.45",YES
3,5FRYD4H4XGB006999251027265521,201809,5FRYD4H4XGB006999,NaN,NaT,NaT,NaT,NaT,nan,NaN,NaN,nan,NaN,nan,nan,200215NB043,NaN,~,~,NaN,NaN,251027,265521,C/S TRANSMISSION FLUID IS LEAKING,2018-09-07,2018-09-18,2015-02-23,2015-0

### Save voucher data as Excel file:

In [80]:
today = datetime.today()

In [81]:
base_dir = "//mmpapp02/mq_db/wrp/ZF_ATM_WRP_Project/automated_vouchers/outputs/excel/NorthAmericanPlants/Fixed"
p = Path(base_dir)
save_dir = p / datetime.strftime(today, "%Y-%m-%d")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [82]:
file_name = 'NA_Plants_Fixed_Voucher_FINAL.xlsx'

In [83]:
voucher_fixed_final.to_excel(save_dir / file_name, index=False)

#### Where Excel file was saved to:

In [84]:
str(save_dir / file_name)

'\\\\mmpapp02\\mq_db\\wrp\\ZF_ATM_WRP_Project\\automated_vouchers\\outputs\\excel\\NorthAmericanPlants\\Fixed\\2019-04-23\\NA_Plants_Fixed_Voucher_FINAL.xlsx'

### Send out Windows notification when complete

In [85]:
toaster = ToastNotifier()
toaster.show_toast("Papermill Orchestration Status",
                   "Successfully created fixed voucher",
                   icon_path="images/honda_logo.ico",
                   duration=5)

True